# EPC EXTENT

import geopandas as gpd
import os
from shapely.geometry import Polygon
import numpy as np

xmin,ymin,xmax,ymax =  tindex.total_bounds
width = 5000
height = 5000
rows = int(np.ceil((ymax-ymin) /  height))
cols = int(np.ceil((xmax-xmin) / width))
XleftOrigin = xmin
XrightOrigin = xmin + width
YtopOrigin = ymax
YbottomOrigin = ymax- height
polygons = []
for i in range(cols):
    Ytop = YtopOrigin
    Ybottom =YbottomOrigin
    for j in range(rows):
        polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
        Ytop = Ytop - height
        Ybottom = Ybottom - height
    XleftOrigin = XleftOrigin + width
    XrightOrigin = XrightOrigin + width

grid = gpd.GeoDataFrame({'geometry':polygons}, crs=tindex.crs)
grid = grid[grid.intersects(tindex.unary_union)]

import os
import geopandas as gpd
from shapely.geometry import Polygon
tindex_loc = "../EPCExtent_30cm/Elevation_80cmNPS/EPCtindex.gpkg"
if not os.path.exists(tindex_loc):
    create_tindex_cmd = f"""find ../EPCExtent_30cm/Elevation_80cmNPS/RGB/Full/ -iname "*.laz" | pdal tindex create --fast_boundary ../EPCExtent_30cm/Elevation_80cmNPS/EPCtindex.gpkg -f "GPKG" --t_srs "EPSG:2868" --stdin --lyr_name pdal"""
    os.system(create_tindex_cmd)
tindex = gpd.read_file(tindex_loc)

# create subtile index since single LAZ files are almost a Gig
nrows = 3
ncols = 3
polygons = []
origin_files = []
indexNums = []
for i, row in tindex.iterrows():
    geom = row.geometry
    xmin,ymin,xmax,ymax = geom.bounds
    width = xmax-xmin
    height=ymax-ymin
    
    target_width = width/ncols
    target_height=height/nrows

    index=0
    for y in range(nrows):
        ymin_s = ymin + (target_height*y)
        ymax_s = ymin + (target_height*(y+1))
        for x in range(ncols):
            index+=1
            xmin_s = xmin + (target_width*x)
            xmax_s = xmin + (target_width*(x+1))
            indexNums.append(index)
            origin_files.append(os.path.abspath(row.location))
            polygons.append(Polygon([(xmin_s, ymax_s), (xmax_s, ymax_s), (xmax_s, ymin_s), (xmin_s, ymin_s)]))
            
subTileIndex = gpd.GeoDataFrame({'originFile':origin_files, "subIndex":indexNums,'geometry':polygons}, crs=tindex.crs)
subTileIndex.to_file("../EPCExtent_30cm/Elevation_80cmNPS/subTileIndex.gpkg", driver="GPKG")
subTileIndex.head(10)

subTileSample = subTileIndex.sample(int(len(subTileIndex)*0.03))
subTileSample.to_file("../EPCExtent_30cm/Elevation_80cmNPS/subTileSample.gpkg", driver="GPKG")

subTileIndex['quads'] = subTileIndex.originFile.apply(lambda f: os.path.basename(f)[:-4])
# make sure that quads used in the downtown area are included in our sample set
downtown_quads = ["E0960_N470", "E0960_N490", "E0980_N430", "E0980_N450", "E0980_N470", "E0980_N490", "E1000_N450", "E1000_N470", "E1020_N450", "E1020_N470"]
must_quads = subTileIndex[subTileIndex.quads.isin(downtown_quads)]
must_quads = must_quads.sample(int(len(must_quads)*0.2))

In [ ]:
import os
import pdal
import gdal
import geopandas as gpd
from datetime import datetime
import json
import rasterio as rio
from joblib import Parallel, delayed
from glob import glob


temp_dir = os.path.join(os.path.abspath("../"), 'temp')
os.makedirs(temp_dir, exist_ok=True)

json_pipe_base = """ { "pipeline": [ %s ] } """


"""
Scalar: Scaling value
    Increase this parameter for terrains with lots of height variation.
    
Slope: slope parameter
    Which is a measure of “slope tolerance”. Increase this parameter for terrains with lots of 
    height variation. Should be set to something higher than 0.1 and not higher than 1.2.
    
Threshold:  Elevation threshold
    Set this parameter to the minimum height (in meters) that you expect non-ground objects to be.
    
Window: Window radius parameter
    Corresponds to the size of the largest feature (building, trees, etc.) to be removed.
    Should be set to a value higher than 10.
    
Threshold option has the biggest impact on results (per OpenDroneMap docs)
"""


filterCrop = """
    {
        "type":"filters.crop",
        "bounds":"([%s, %s],[%s, %s])",
        "a_srs":"EPSG:2868"   
    }"""
filterGround = """
    {
        "type":"filters.range",
        "limits":"Classification[2:2]"
    }"""
writeLAZ = """
    {
        "type":"writers.las",
        "compression":"laszip",
        "filename":"%s"
    }"""
writeTIFF = """
    {
        "type":"writers.gdal",
        "filename": "%s",
        "resolution": %s,
        "window_size": %s,
        "nodata": 0,
        "output_type": "idw",
        "gdalopts":"TILED=YES, COMPRESS=LZW"
    }"""


laz_folder = r"../EPCExtent_30cm/Elevation_80cmNPS/RGB/Full"

dtm_folder = r"../EPCExtent_30cm/Elevation_80cmNPS/DTM80cm"
ground_folder = r"../EPCExtent_30cm/Elevation_80cmNPS/RGB/ground"
os.makedirs(dtm_folder, exist_ok=True)
os.makedirs(ground_folder, exist_ok=True)

inout_files = {}

for f in os.listdir(laz_folder):
    if f.endswith(".laz"):
        infile = os.path.join(laz_folder, f)
        outfile = os.path.join(dtm_folder, f.replace(".laz", "_DTM80cm.tif"))
        inout_files[infile] = outfile
        
        
def createDTM(row, laz_folder, dtm_folder, params, overwrite=False):
    infile = row.originFile
    fname = os.path.basename(infile)
    #suffix = f"Sc{params['scalar']}Sl{params['slope']}Th{params['threshold']}WS{params['win_size']}"
    suffix = os.path.basename(dtm_folder)
    
    #outlaz = os.path.join(laz_folder, fname.replace(".laz", f"_{row.subIndex}_{suffix}.laz"))
    outtiff = os.path.join(dtm_folder, fname.replace(".laz", f"_{row.subIndex}_{suffix}.tif"))
    
    if os.path.exists(outtiff) and overwrite == False:
        return
    
    filterSMRF = """
    {
        "type": "filters.smrf",
        "returns":"first,last,intermediate,only",
        "cell":%s,
        "scalar": %s,
        "slope": %s,
        "threshold": %s,
        "window": %s     
    }"""
    
    geom = row.geometry.buffer(params['win_size'])
    xmin,ymin,xmax,ymax = geom.bounds
    
    #if os.path.exists(outlaz) and not overwrite:
    #    return
        #pipeline_json = """ "%s" """ % outlaz
    if not os.path.exists(outtiff):
        pipeline_json = """ "%s" """ % infile
        pipeline_json += ',' + filterCrop % (xmin,xmax,ymin,ymax)
        pipeline_json += ',' + filterSMRF % (params['resolution'],params['scalar'], params['slope'], params['threshold'], params['win_size'])
        #pipeline_json += ',' + filterPMF % (8, 0.15, 40, 50, 1.5)
        pipeline_json += ',' + filterGround
        #pipeline_json += ',' + writeLAZ % (outlaz)
        pipeline_json += ',' + writeTIFF % (outtiff, params['resolution'], 0)
    else:
        return outtiff
        
        
    pipeline_json = json_pipe_base % pipeline_json.replace("\\","/")
    
    pipeline = pdal.Pipeline(pipeline_json)
    pipeline.loglevel = 8

    start = datetime.now()
    try:
        pipeline.execute()
    except:
        print(f"Failed for {outtiff}")
    end = datetime.now()

    print(f"{datetime.now()}\t-\tFinished with {os.path.basename(outtiff)} - {end-start} elapsed")
    
    return outtiff
    
print("\nBeginning DTM Generations for {} QQuads\n".format(len(inout_files)))

res = round(0.8/0.3048,5)

# default scalar is 1.25
#scalars = (0, 0.5, 1, 1.5)
scalars = (0, 0.25, 0.5, 1, 1.5, 3)
# default slope is 0.15
#slopes = (0.01,0.10,0.25)
slopes = (0.001, 0.005, 0.01, 0.10, 0.25)
# default threshold is 0.5
#thresholds = (0.5, 1, 1.5)
thresholds = (0.1, 0.25, 0.5, 1, 1.5, 3)
#default window is 18
#windows = [100]
windows = [100,200]

tindex_2015_loc = "/media/ben/PAG2015Elev/PAG2015DEM_index.gpkg"
tindex_2015 = gpd.read_file(tindex_2015_loc).to_crs("epsg:2868")

tindex_loc = "../EPCExtent_30cm/Elevation_80cmNPS/EPCtindex.gpkg"
if not os.path.exists(tindex_loc):
    create_tindex_cmd = f"""find ../EPCExtent_30cm/Elevation_80cmNPS/RGB/Full/ -iname "*.laz" | pdal tindex create --fast_boundary ../EPCExtent_30cm/Elevation_80cmNPS/EPCtindex.gpkg -f "GPKG" --t_srs "EPSG:2868" --stdin --lyr_name pdal"""
    os.system(create_tindex_cmd)
tindex = gpd.read_file(tindex_loc)

subTileIndex_loc = "../EPCExtent_30cm/Elevation_80cmNPS/subTileIndex.gpkg"
if not os.path.exists(subTileIndex_loc):
    # create subtile index since single LAZ files are almost a Gig
    nrows = 3
    ncols = 3
    polygons = []
    origin_files = []
    indexNums = []
    for i, row in tindex.iterrows():
        geom = row.geometry
        xmin,ymin,xmax,ymax = geom.bounds
        width = xmax-xmin
        height=ymax-ymin

        target_width = width/ncols
        target_height=height/nrows

        index=0
        for y in range(nrows):
            ymin_s = ymin + (target_height*y)
            ymax_s = ymin + (target_height*(y+1))
            for x in range(ncols):
                index+=1
                xmin_s = xmin + (target_width*x)
                xmax_s = xmin + (target_width*(x+1))
                indexNums.append(index)
                origin_files.append(os.path.abspath(row.location))
                polygons.append(Polygon([(xmin_s, ymax_s), (xmax_s, ymax_s), (xmax_s, ymin_s), (xmin_s, ymin_s)]))

    subTileIndex = gpd.GeoDataFrame({'originFile':origin_files, "subIndex":indexNums,'geometry':polygons}, crs=tindex.crs)
    subTileIndex.to_file(subTileIndex_loc, driver="GPKG")
subTileIndex = gpd.read_file(subTileIndex_loc)

#make sure sub tiles intersect 2015 data for comparison
subTileIndex = subTileIndex[subTileIndex.within(tindex_2015.unary_union.buffer(-500))]

#sample_subtiles
subTileSample_loc = "../EPCExtent_30cm/Elevation_80cmNPS/subTileSample.gpkg"
if not os.path.exists(subTileSample_loc):
    print("Creating new sub tile sample set")
    subTileSample = subTileIndex.sample(int(len(subTileIndex)*0.03))
    subTileSample.to_file(subTileSample_loc, driver="GPKG")
subTileSample = gpd.read_file(subTileSample_loc)
    
centralTucsonSample_loc = "../EPCExtent_30cm/Elevation_80cmNPS/centralTucson_sample.gpkg"
if not os.path.exists(centralTucsonSample_loc):
    subTileIndex['quads'] = subTileIndex.originFile.apply(lambda f: os.path.basename(f)[:-4])
    # make sure that quads used in the downtown area are included in our sample set
    downtown_quads = ["E0960_N470", "E0960_N490", "E0980_N430", "E0980_N450", "E0980_N470", "E0980_N490", "E1000_N450", "E1000_N470", "E1020_N450", "E1020_N470"]
    centralTucsonSample = subTileIndex[subTileIndex.quads.isin(downtown_quads)]
    centralTucsonSample = centralTucsonSample.sample(int(len(centralTucsonSample)*0.1))
    centralTucsonSample.to_file(centralTucsonSample_loc, driver="GPKG")
centralTucsonSample = gpd.read_file(centralTucsonSample_loc)

total_possibilities = len(scalars) * len(slopes) * len(thresholds) * len(windows)
combo_count = 0

for name,df in {"Central Tucson" : centralTucsonSample, "Regional Sample" : subTileSample}.items():
    print(f"Starting on {name}. {len(name)} total tiles")
    for sci, sc in enumerate(scalars):
        for sli, sl in enumerate(slopes):
            for ti, thresh in enumerate(thresholds):
                for wi, win in enumerate(windows):
                    
                    combo_count += 1
                    print(f"Starting combo {combo_count} of {total_possibilities}")
                    
                    parameters=dict(
                        scalar=sc,
                        slope=sl,
                        threshold=thresh,
                        win_size=win,
                        resolution=res
                    )
                    suffix = f"Sc{parameters['scalar']}Sl{parameters['slope']}Th{parameters['threshold']}WS{parameters['win_size']}"
                    out_DTMFolder = os.path.join(dtm_folder,suffix)
                    out_groundFolder = os.path.join(ground_folder, suffix)
                    os.makedirs(out_DTMFolder, exist_ok=True)
                    os.makedirs(out_groundFolder, exist_ok=True)
                    
                    t1 = datetime.now()
                    print(f"Starting at {t1} with params\n\tScalar:  {sc}\n\tSlope: {sl}\n\tThreshold: {thresh}\n\tWindows Size:{win}")

                    try:

                        json_pipes = Parallel(n_jobs=4, verbose=5, backend="loky")(delayed(
                            createDTM)(row, out_groundFolder, out_DTMFolder, parameters, overwrite=False) for i, row in df.iterrows())

                    except:
                        json_pipes = Parallel(n_jobs=4, verbose=5, backend="loky")(delayed(
                            createDTM)(row, out_groundFolder, out_DTMFolder, parameters, overwrite=False) for i, row in df.iterrows())

                    print(f"Finished in {datetime.now()-t1}")


Beginning DTM Generations for 326 QQuads

Starting on Central Tucson. 14 total tiles
Starting combo 1 of 360
Starting at 2020-07-10 15:55:58.195908 with params
	Scalar:  0
	Slope: 0.001
	Threshold: 0.1
	Windows Size:100


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   9 | elapsed:  2.7min remaining:  3.3min
[Parallel(n_jobs=4)]: Done   6 out of   9 | elapsed:  5.0min remaining:  2.5min


Finished in 0:07:27.559134
Starting combo 2 of 360
Starting at 2020-07-10 16:03:25.755612 with params
	Scalar:  0
	Slope: 0.001
	Threshold: 0.1
	Windows Size:200


[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed:  7.5min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   9 | elapsed:  3.9min remaining:  4.9min
/home/ben/anaconda3/envs/geospatial/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
print(f"Starting at {t1} with params\n\tScalar:  {sc}\n\tSlope: {sl}\n\tThreshold: {thresh}")
                
for i, row in centralTucsonSample.iterrows():
    json = createDTM(row, ground_folder, dtm_folder, parameters, overwrite=True)
    break
print(f"Finished in {datetime.now()-t1}")

In [ ]:
dist_loc = "../EPCExtent_30cm/Elevation_80cmNPS/HAG_NED_80cm_DTG"
distance_files = glob(f"{dist_loc}/*.tif")

params=parameters.copy()
parameters.update(scalar=1.5, slope=0.15, threshold=0.5)

dtg_breaks = (20,40,100,200,500)
params = parameters.copy()

infile = row.originFile
fname = os.path.basename(infile)
suffix = f"Sc{params['scalar']}Sl{params['slope']}Th{params['threshold']}"
print(suffix)
outlaz = os.path.join(ground_folder, fname.replace(".laz", f"_{row.subIndex}_{suffix}.laz"))
outtiff = os.path.join(dtm_folder, fname.replace(".laz", f"_{row.subIndex}_{suffix}.tif"))

for f in distance_files:
    if fname.replace(".laz","") in f:
        dtg_file = f
        break

dimension = "GpsTime"
filterColorization = """{
      "type":"filters.colorization",
      "dimensions":"%s",
      "raster":"%s"
}"""
filterSMRF = """
{
    "type": "filters.smrf",
    "returns":"first,last,intermediate,only",
    "cell":%s,
    "scalar": %s,
    "slope": %s,
    "threshold": %s,
    "ignore":"%s",
    "window": %s     
}"""
filterAssign = """
{
      "type":"filters.assign",
      "assignment": "%s[:]=0"
}"""


geom = row.geometry.buffer(100)
xmin,ymin,xmax,ymax = geom.bounds

overwrite=True
if os.path.exists(outlaz) and not overwrite:
    print("ok")
    #pipeline_json = """ "%s" """ % outlaz
else:
    pipeline_json = """ "%s" """ % infile
    #crop input file to bounds of subsection
    pipeline_json += ',' + filterCrop % (xmin,xmax,ymin,ymax)
    #assign distance to ground values to existing dimension. Using GpsTime
    pipeline_json += ',' + filterAssign % dimension
    #extract values of DTG raster to laz
    pipeline_json += ',' + filterColorization % (dimension, dtg_file)
    
    pipeline_json += ',' + filterSMRF % (params['resolution'],params['scalar'], params['slope'], params['threshold'], f"{dimension}![:10]", 15)
    # add SMRF filters for each window size
    for i in range(len(dtg_breaks)-1):
        dtg_break = dtg_breaks[i]
        low = dtg_break
        if i != len(dtg_breaks) -2:
            high = dtg_breaks[i+1]
        else:
            high = ""
        rng = f"{low}:{high}"
                                         
        bs = 10 if dtg_break == 20 else 20
        win_size = high + bs if dtg_breaks[-2] != dtg_break else low + bs
        if i == len(dtg_breaks):
            break
        pipeline_json += ',' + filterSMRF % (params['resolution'],params['scalar'], params['slope'], params['threshold'], f"{dimension}![{rng}]", win_size+20)
        #pipeline_json += ',' + filterPMF % (8, 0.15, 40, 50, 1.5)
    pipeline_json += ',' + filterGround
    #pipeline_json += ',' + writeLAZ % (outlaz)

if not os.path.exists(outtiff):
    pipeline_json += ',' + writeTIFF % (outtiff, params['resolution'],200)

pipeline_json = json_pipe_base % pipeline_json.replace("\\","/")

pipeline = pdal.Pipeline(pipeline_json)
pipeline.loglevel = 8

start = datetime.now()
try:
    pipeline.execute()
except:
    print(f"Failed for {outlaz}")
end = datetime.now()

print(f"{datetime.now()}\t-\tFinished with {os.path.basename(outlaz)} - {end-start} elapsed")


In [ ]:
with open("smrfMod.json", "w") as dst:
    dst.write(pipeline_json)

In [ ]:
params = parameters.copy()
params


    
    
for i, row in subTileIndex.iterrows():
    infile = row.originFile
    file = os.path.basename(infile)
    outlaz = os.path.join(ground_folder, file.replace(".laz", f"_{row.subIndex}.laz"))
    outtiff = os.path.join(dtm_folder, file.replace(".laz", f"_{row.subIndex}.tif"))
    inout_files[infile] = outfile

            
    t1=datetime.now()
    
    #json_pipe = createDTM(infile, outfile, subset_bounds, parameters, overwrite=False)
    if not os.path.exists(outfile):
        try:
            print(f"Starting on {outfile} at {t1}")
            json_pipes = Parallel(n_jobs=5, verbose=20, backend="loky", max_nbytes=None)(delayed(
                createDTM)(infile, outfile, subbounds, i, parameters, overwrite=False) for i, subbounds in enumerate(subset_bounds))
            t2=datetime.now()
            print(f"Created file {outfile} in {t2-t1}")
        except:
            print(f"Failed for {outfile}. Error")
            json_pipes = Parallel(n_jobs=2, verbose=20, backend="loky", max_nbytes=None)(delayed(
                createDTM)(infile, outfile, subbounds, i, parameters, overwrite=False) for i, subbounds in enumerate(subset_bounds))
            t2=datetime.now()
            print(f"Created file {outfile} in {t2-t1}")
           
    
    
print("\n\n...FINISHED\n\n")

In [ ]:
"Done"

- 4/25 of a tile at window 30 uses up to 85% - tile took 25.5min, cell = 1
- 4/25 of a tile at window 10 uses up to 85% - tile took 19.3min, cell = 1
- 4/25 of a tile at window 10 uses up to 85% - tile took 22min, cell = 1

- 5/25 of a tile at window 30 uses up to 85% - tile took 7.5 min, cell = 2.62
- 2/4 of tile at window 30 uses up to 50% - tile took 4.9min, cell = 2.62
- 3/4 of tile at window 30 uses up to 78% - tile took 4.8min, cell = 2.62
- 4/8 of tile at window 30 uses up to 78% - tile took 3.7min, cell = 2.62
- 5/9 of tile at window 30 uses up to 83% - tile took 3.6min, cell = 2.62
- 5/10 of tile at window 30 uses up to 90% - tile took 3.5min, cell = 2.62

--------------------------------

In [ ]:
raw_files = ["0300", "0219", "0218", "0025", "0024", "0022"]

crop_pipe = """{ "pipeline": [ 
    "%s",
	{
        "type":"filters.crop",
        "bounds":"([979919.82, 990079.75],[499900.12, 510060.05])"
    },
    {
        "type":"writers.las",
        "compression":"laszip",
        "filename":"./ground/%s"
    }   
 ] }"""
merge_pipe = """{ "pipeline": [
    %s
    {
        "type":"filters.merge"
    },
    {
        "type":"filters.outlier",
        "method":"statistical",
        "mean_k":12,
        "multiplier":2.2
    },
    
    {
        "type":"filters.range",
        "limits":"Classification[:6], Classification[8:]"
    },
    {
        "type": "filters.smrf",
        "cell": 25,
        "dir": "P:/GISLibrary/OrthoPhotos/Ortho2017/UrbanExtent_15cm/Elevation/RGB/temp",
        "returns":"first,last,intermediate,only",
        "ignore": "Classification[7:7]",
        "scalar": 1.0,
        "slope": 0.18,
        "threshold": 1.5,
        "window": 30     
    },
    {
        "type":"filters.range",
        "limits":"Classification[2:2]"
    },
    {
        "type":"writers.las",
        "filename": "%s",
        "compression":"laszip"
    }
 ] }"""

ofiles = ""
for f in raw_files:
    fname =  "St_" + f + ".las"
    fpath = os.path.join(r"G:\GISLibrary\OrthoPhotos\Ortho2015\Final_Shipped_USGS_Feb2016\QL2\Raw_Point_Cloud_16bit", fname)
    opath = os.path.join(r"P:\GISLibrary\OrthoPhotos\Ortho2017\UrbanExtent_15cm\Elevation\RGB\ground", fname.replace(".las",".laz"))
    pipeline_json = crop_pipe % (fpath.replace("\\","/"), opath.replace("\\", "/"))
    #print(pipeline_json)
    
    pipeline = pdal.Pipeline(pipeline_json)
    pipeline.loglevel = 8
    start = datetime.now()
    #pipeline.execute()
    end = datetime.now()
    print(f"{end} \t- Crop of {fname} done in {end-start}")
    ofiles += '"' + opath.replace('\\','/') + '", '
            
        
ofiles = ofiles[:-1]
ofile = r"P:/GISLibrary/OrthoPhotos/Ortho2017/UrbanExtent_15cm/Elevation/RGB/ground/2015QL2UnClassified_985503.laz"
pipeline_json = merge_pipe % (ofiles, ofile)

pipeline = pdal.Pipeline(pipeline_json)
start = datetime.now()
pipeline.execute()
end = datetime.now()
print(f"{end} \t- Merge of files done in {end-start}")


In [ ]:
merge_pipe = """{ "pipeline": [
    "%s",
    {
        "type": "filters.smrf",
        "cell": 25,
        "dir": "P:/GISLibrary/OrthoPhotos/Ortho2017/UrbanExtent_15cm/Elevation/RGB/temp",
        "returns":"last,only",
        "ignore": "Classification[7:7]",
        "scalar": 1.0,
        "slope": 0.15,
        "threshold": 1,
        "window": 30     
    },
    {
        "type":"filters.range",
        "limits":"Classification[2:2]"
    },
    {
        "type":"writers.las",
        "filename": "%s",
        "compression":"laszip"
    }
 ] }"""

ifile = r"P:/GISLibrary/OrthoPhotos/Ortho2017/UrbanExtent_15cm/Elevation/RGB/ground/2015QL2UnClassified_985503.laz"
ofile = r"P:/GISLibrary/OrthoPhotos/Ortho2017/UrbanExtent_15cm/Elevation/RGB/ground/2015QL2UnClassified_SMRF_985503.laz"
pipeline_json = merge_pipe % (ifile, ofile)

pipeline = pdal.Pipeline(pipeline_json)
start = datetime.now()
pipeline.execute()
end = datetime.now()
print(f"{end} \t- Merge of files done in {end-start}")


In [ ]:
pipeline = pdal.Pipeline(info_pipe % (inlas, x, y, 1))
pipeline.loglevel = 1
start = datetime.now()
pipeline.execute()
end = datetime.now()
md_dict = json.loads(pipeline.metadata)
elevation = md_dict['metadata']['filters.info']['points']['point']['Z']
print(f"{elevation}\t-\tFinished - {end-start} elapsed")

In [ ]:
json.loads(pipeline.metadata)